In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
import torch.nn.functional as F
import pandas as pd
from torchinfo import summary

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
batch_size = 4
num_epochs = 4
learning_rate = .001
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [6]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data  = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_data,  batch_size=batch_size, shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [7]:
example = iter(train_loader)
sample, label = example._next_data()
print(sample.shape)

torch.Size([4, 3, 32, 32])


In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16*5*5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out



In [10]:
model = ConvNet().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

summary(model, (4,3,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [4, 10]                   --
├─Conv2d: 1-1                            [4, 6, 28, 28]            456
├─MaxPool2d: 1-2                         [4, 6, 14, 14]            --
├─Conv2d: 1-3                            [4, 16, 10, 10]           2,416
├─MaxPool2d: 1-4                         [4, 16, 5, 5]             --
├─Linear: 1-5                            [4, 128]                  51,328
├─Linear: 1-6                            [4, 64]                   8,256
├─Linear: 1-7                            [4, 10]                   650
Total params: 63,106
Trainable params: 63,106
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.64
Input size (MB): 0.05
Forward/backward pass size (MB): 0.21
Params size (MB): 0.25
Estimated Total Size (MB): 0.51

In [9]:
n_total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%2000 == 0:
            print(f"epoch {epoch} / {num_epochs}, step {i} / {n_total_step}, loss = {loss.item():.4f}")

epoch 0 / 4, step 1999 / 12500, loss = 2.2887
epoch 0 / 4, step 3999 / 12500, loss = 2.2881
epoch 0 / 4, step 5999 / 12500, loss = 2.3507
epoch 0 / 4, step 7999 / 12500, loss = 1.9519
epoch 0 / 4, step 9999 / 12500, loss = 2.5861
epoch 0 / 4, step 11999 / 12500, loss = 2.1568
epoch 1 / 4, step 1999 / 12500, loss = 1.3649
epoch 1 / 4, step 3999 / 12500, loss = 2.2864
epoch 1 / 4, step 5999 / 12500, loss = 1.7582
epoch 1 / 4, step 7999 / 12500, loss = 1.4843
epoch 1 / 4, step 9999 / 12500, loss = 2.1199
epoch 1 / 4, step 11999 / 12500, loss = 1.1735
epoch 2 / 4, step 1999 / 12500, loss = 1.5722
epoch 2 / 4, step 3999 / 12500, loss = 2.5826
epoch 2 / 4, step 5999 / 12500, loss = 1.6096
epoch 2 / 4, step 7999 / 12500, loss = 1.0406
epoch 2 / 4, step 9999 / 12500, loss = 1.9668
epoch 2 / 4, step 11999 / 12500, loss = 1.4972


KeyboardInterrupt: 

In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)

        _, prediction = torch.max(output, 1)

        n_samples += labels.size(0)

        n_correct += (prediction == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred  = prediction[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f"accuracy:{acc}")

    for i in range(10):
        acc = 100 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}')

accuracy:46.38
Accuracy of plane:33.1
Accuracy of car:58.8
Accuracy of bird:26.1
Accuracy of cat:33.4
Accuracy of deer:29.7
Accuracy of dog:50.9
Accuracy of frog:59.7
Accuracy of horse:59.5
Accuracy of ship:57.4
Accuracy of truck:55.2
